<a href="https://colab.research.google.com/github/nagolinc/notebooks/blob/main/cycle_diffsuion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install wandb

In [ ]:
!wandb login

# you will need to create a project in wandb and edit these variables!

In [ ]:
%env WANDB_PROJECT=<your project name>
%env WANDB_ENTITY=<your wandb username>

# install a ton of stuff

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
!git clone https://github.com/CompVis/taming-transformers.git

In [ ]:
cd taming-transformers/

In [ ]:
pip install -e .

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!pip install blobfile

In [ ]:
!pip install OmegaConf

In [ ]:
!pip install einops

In [ ]:
!pip install pytorch_lightning

In [ ]:
!pip install kornia

In [ ]:
cd /content

In [ ]:
!git clone https://github.com/ChenWu98/cycle-diffusion.git

In [ ]:
cd cycle-diffusion

In [ ]:
cd ckpts/


In [ ]:
mkdir stable_diffusion


# You need a copy of sd-v1-4.ckpt (I save one in my google drive for this)

In [ ]:
cp /content/drive/MyDrive/AI/models/sd-v1-4.ckpt /content/cycle-diffusion/ckpts/stable_diffusion

In [ ]:
cd /content/cycle-diffusion

# And generate our image

In [ ]:
!wget https://pbs.twimg.com/media/FfV8d2yXEAIgIKa?format=png

In [ ]:
#edit data/translate-text.json
with open("/content/cycle-diffusion/data/translate-text.json",'w') as f:
  f.write("""[
    {
        "encode_text": "A photo of Barack Obama",
        "decode_text": "A photo of Barack Obama smiling big grin",
        "img_path": "/content/cycle-diffusion/FfV8d2yXEAIgIKa?format=png"
    }
]""")

In [ ]:
with open("/content/cycle-diffusion/config/experiments/translate_text2img256_stable_diffusion_stochastic_custom_1.cfg",'w') as f:
  f.write("""[model]
name = text_unsupervised_translation

[gan]
gan_type = SDStochasticText
source_model_type = sd-v1-4.ckpt
custom_steps = 99
white_box_steps = 100
eta = 0.1
encoder_unconditional_guidance_scales = [1]
#this is how strongly the prompt will effect the image (3-5-ish is a good value)
decoder_unconditional_guidance_scales = [3]
#increase number of generations (only the best will be output)
n_trials = 1
#how may diffusion steps to skip (higher=fine grained changes only)
skip_steps = [10]

[raw_data]
upsample_temp = 1
#don't change this!
range = [0, 500]

[arg_paths]
translate = tasks/translate_text512.cfg

[PriorZEnergy]
weight = 1

[evaluation]
evaluator_program = multi_task

[visualization]
visualizer_program = multi_image""")

In [ ]:
%env CUDA_VISIBLE_DEVICES=0
%env RUN_NAME=translate_text2img256_stable_diffusion_stochastic_custom_1
%env SEED=42
!python -m torch.distributed.launch --nproc_per_node 1 --master_port 1405 main.py --seed $SEED \
 --cfg experiments/$RUN_NAME.cfg \
 --run_name $RUN_NAME$SEED \
 --logging_strategy steps \
 --logging_first_step true \
 --logging_steps 4 \
 --evaluation_strategy steps \
 --eval_steps 50 \
 --metric_for_best_model CLIPEnergy \
 --greater_is_better false \
 --save_strategy steps \
 --save_steps 50 \
 --save_total_limit 1 \
 --load_best_model_at_end \
 --gradient_accumulation_steps 4 \
 --num_train_epochs 0 \
 --adafactor false \
 --learning_rate 1e-3 \
 --do_eval \
 --output_dir output/$RUN_NAME$SEED \
 --overwrite_output_dir \
 --per_device_train_batch_size 1 \
 --per_device_eval_batch_size 4 \
 --eval_accumulation_steps 4 \
 --ddp_find_unused_parameters true \
 --verbose true